In [3]:
import numpy as np
import json
import os
import copy

In [2]:
from create_new_project import *

location = '/media/storage/ryan/'
name = 'mabe_challenge'

create_new_project(location, name)

A project named mabe_challenge already exists at this location. Please delete it or choose a different name.


In [6]:
train = np.load('../data/train.npy',allow_pickle=True).item()
test = np.load('../data/test-release.npy',allow_pickle=True).item()
sample_submission = np.load('../data/sample-submission.npy',allow_pickle=True).item()

In [7]:
def pad_bbox(bbox_1, bbox_2, min_b, max_b, pad_percentage=0.0):
    '''Pad the edges of a bounding box by a given amount.'''
    # Get the length of the side
    bbox_l = (bbox_2*1.0 - bbox_1*1.0)

    # Compute the tentative padding length
    pad_tentative = pad_percentage*bbox_l/2.

    # Compute the maximum feasible padding amounts
    right_max = max_b - bbox_2
    left_max = bbox_1 - min_b

    # Restrict the pad to what's feasible.
    pad = np.minimum(right_max, left_max)
    pad = np.minimum(pad, pad_tentative)

    # Add the padding
    new_bbox_1 = bbox_1 - pad
    new_bbox_2 = bbox_2 + pad

    # Rectify the bounding boxes to be feasible (Shouldn't be necessary)
    new_bbox_1 = (np.maximum(new_bbox_1, min_b))
    new_bbox_2 = (np.minimum(new_bbox_2, max_b))

    return [new_bbox_1, new_bbox_2]

def convert_npy_to_json(data):
    converted = {}
    if "vocabulary" in data.keys():
        converted["vocabulary"] = data["vocabulary"]
    else:
        converted["vocabulary"] = {'attack': 0, 'investigation': 1, 'mount': 2, 'other': 3};
    converted["sequences"] = {}
    converted["sequences"]["mabe_challenge"] = data["sequences"]
    for seq_id in data["sequences"]:
        keypoints = data["sequences"][seq_id]["keypoints"]
        np.shape(np.concatenate((np.amin(keypoints, axis=3), np.amax(keypoints, axis=3)), axis=2))
        bbox = np.concatenate((np.amin(keypoints, axis=3), np.amax(keypoints, axis=3)), axis=2)
        bbox[:, :, (0, 2)] /= 1024
        bbox[:, :, (1, 3)] /= 570

        for i in range(bbox.shape[0]):
            for j in range(bbox.shape[1]):
                bbox[i, j, (0, 2)] = pad_bbox(bbox[i, j, 0], bbox[i, j, 2], 0, 1, 0.25)
                bbox[i, j, (1, 3)] = pad_bbox(bbox[i, j, 1], bbox[i, j, 3], 0, 1, 0.25)
        converted["sequences"]["mabe_challenge"][seq_id]["bbox"] = np.moveaxis(bbox,0, -1).tolist()
        converted["sequences"]["mabe_challenge"][seq_id]["keypoints"] = data["sequences"][seq_id]["keypoints"].tolist()
        if "annotations" in data["sequences"][seq_id].keys():
            converted["sequences"]["mabe_challenge"][seq_id]["annotations"] = data["sequences"][seq_id]["annotations"].tolist()
        converted["sequences"]["mabe_challenge"][seq_id] = [converted["sequences"]["mabe_challenge"][seq_id]]
    return converted
        
def get_nth_key(dictionary, n=0):
    if n < 0:
        n += len(dictionary)
    for i, key in enumerate(dictionary.keys()):
        if i == n:
            return key
    raise IndexError("dictionary index out of range") 


In [10]:
train_json = convert_npy_to_json(train)

with open('/media/storage/ryan/mabe_challenge/behavior/behavior_jsons/train_data.json', 'w') as f:
    json.dump(train_json, f)

In [6]:
len(test["sequences"].keys())

458

In [8]:
batch_size = 20

start_index = 0
count = 0

while(start_index < len(test["sequences"].keys())):
    print(count)
    end_index = start_index + batch_size
    if end_index > len(test["sequences"].keys()):
        end_index = len(test["sequences"].keys())
    
    test_subset = copy.deepcopy(test)
    temp_sequences = {}
    for index in range(start_index, end_index):
        key = get_nth_key(test_subset["sequences"], index)
        temp_sequences[key] = copy.deepcopy(test_subset["sequences"][key])
    
    test_subset["sequences"] = temp_sequences
    
    test_json = convert_npy_to_json(test_subset)
    with open('/media/storage/ryan/mabe_challenge/behavior/behavior_jsons/test_' + str(count) + '_data.json', 'w') as f:
        json.dump(test_json, f)
    
    count += 1
    start_index = end_index

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22


In [11]:
test_json = convert_npy_to_json(test)

with open('/media/storage/ryan/mabe_challenge/behavior/behavior_jsons/test_data.json', 'w', 'w') as f:
    json.dump(test_json, f)

In [9]:
with open('/media/storage/ryan/mabe_challenge/behavior/behavior_jsons/train_data.json') as f:
    t = json.load(f)

In [14]:
t["sequences"]["mabe_challenge"] = {'1cac195d39': t["sequences"]["mabe_challenge"]['1cac195d39']}

In [17]:
with open('/media/storage/ryan/mabe_challenge/behavior/behavior_jsons/example_data.json', 'w') as f:
    json.dump(t, f)

In [11]:
test_list = []
for i in range(5, 23):
    test_list.append('test_' + str(i))

In [1]:
from behavior_classification import MARS_feature_extractor

project_path = '/media/storage/ryan/mabe_challenge'

MARS_feature_extractor.extract_features(project_path, sets_to_process = ["example"])
#MARS_feature_extractor.list_features(project_path)

example video 1/1: 1cac195d39 clip 1/1
['top_m0_phi', 'top_m0_ori_head', 'top_m0_ori_body', 'top_m0_angle_head_body_l', 'top_m0_angle_head_body_r', 'top_m0_major_axis_len', 'top_m0_minor_axis_len', 'top_m0_axis_ratio', 'top_m0_area_ellipse', 'top_m0_dist_edge_x', 'top_m0_dist_edge_y', 'top_m0_dist_edge', 'top_m0_speed', 'top_m0_speed_centroid', 'top_m0_speed_fwd', 'top_m0_acceleration_head', 'top_m0_acceleration_body', 'top_m0_acceleration_centroid', 'top_m0_nose_x', 'top_m0_nose_y', 'top_m0_right_ear_x', 'top_m0_right_ear_y', 'top_m0_left_ear_x', 'top_m0_left_ear_y', 'top_m0_neck_x', 'top_m0_neck_y', 'top_m0_right_side_x', 'top_m0_right_side_y', 'top_m0_left_side_x', 'top_m0_left_side_y', 'top_m0_tail_base_x', 'top_m0_tail_base_y', 'top_m0_centroid_x', 'top_m0_centroid_y', 'top_m0_centroid_head_x', 'top_m0_centroid_head_y', 'top_m0_centroid_body_x', 'top_m0_centroid_body_y', 'top_m0_dist_nose_right_ear', 'top_m0_dist_nose_left_ear', 'top_m0_dist_nose_neck', 'top_m0_dist_nose_right_sid

In [ ]:
import json
import numpy as np
with open("/media/storage/ryan/mabe_challenge/behavior/behavior_jsons/example_features.json") as f:
    t = json.load(f)

In [4]:
features = ['top_m0_phi', 'top_m0_ori_head', 'top_m0_ori_body', 'top_m0_angle_head_body_l', 'top_m0_angle_head_body_r', 'top_m0_major_axis_len', 'top_m0_minor_axis_len', 'top_m0_axis_ratio', 'top_m0_area_ellipse', 'top_m0_dist_edge_x', 'top_m0_dist_edge_y', 'top_m0_dist_edge', 'top_m0_speed', 'top_m0_speed_centroid', 'top_m0_speed_fwd', 'top_m0_acceleration_head', 'top_m0_acceleration_body', 'top_m0_acceleration_centroid', 'top_m0_nose_x', 'top_m0_nose_y', 'top_m0_right_ear_x', 'top_m0_right_ear_y', 'top_m0_left_ear_x', 'top_m0_left_ear_y', 'top_m0_neck_x', 'top_m0_neck_y', 'top_m0_right_side_x', 'top_m0_right_side_y', 'top_m0_left_side_x', 'top_m0_left_side_y', 'top_m0_tail_base_x', 'top_m0_tail_base_y', 'top_m0_centroid_x', 'top_m0_centroid_y', 'top_m0_centroid_head_x', 'top_m0_centroid_head_y', 'top_m0_centroid_body_x', 'top_m0_centroid_body_y', 'top_m0_dist_nose_right_ear', 'top_m0_dist_nose_left_ear', 'top_m0_dist_nose_neck', 'top_m0_dist_nose_right_side', 'top_m0_dist_nose_left_side', 'top_m0_dist_nose_tail_base', 'top_m0_dist_right_ear_nose', 'top_m0_dist_right_ear_left_ear', 'top_m0_dist_right_ear_neck', 'top_m0_dist_right_ear_right_side', 'top_m0_dist_right_ear_left_side', 'top_m0_dist_right_ear_tail_base', 'top_m0_dist_left_ear_nose', 'top_m0_dist_left_ear_right_ear', 'top_m0_dist_left_ear_neck', 'top_m0_dist_left_ear_right_side', 'top_m0_dist_left_ear_left_side', 'top_m0_dist_left_ear_tail_base', 'top_m0_dist_neck_nose', 'top_m0_dist_neck_right_ear', 'top_m0_dist_neck_left_ear', 'top_m0_dist_neck_right_side', 'top_m0_dist_neck_left_side', 'top_m0_dist_neck_tail_base', 'top_m0_dist_right_side_nose', 'top_m0_dist_right_side_right_ear', 'top_m0_dist_right_side_left_ear', 'top_m0_dist_right_side_neck', 'top_m0_dist_right_side_left_side', 'top_m0_dist_right_side_tail_base', 'top_m0_dist_left_side_nose', 'top_m0_dist_left_side_right_ear', 'top_m0_dist_left_side_left_ear', 'top_m0_dist_left_side_neck', 'top_m0_dist_left_side_right_side', 'top_m0_dist_left_side_tail_base', 'top_m0_dist_tail_base_nose', 'top_m0_dist_tail_base_right_ear', 'top_m0_dist_tail_base_left_ear', 'top_m0_dist_tail_base_neck', 'top_m0_dist_tail_base_right_side', 'top_m0_dist_tail_base_left_side', 'top_m1_phi', 'top_m1_ori_head', 'top_m1_ori_body', 'top_m1_angle_head_body_l', 'top_m1_angle_head_body_r', 'top_m1_major_axis_len', 'top_m1_minor_axis_len', 'top_m1_axis_ratio', 'top_m1_area_ellipse', 'top_m1_dist_edge_x', 'top_m1_dist_edge_y', 'top_m1_dist_edge', 'top_m1_speed', 'top_m1_speed_centroid', 'top_m1_speed_fwd', 'top_m1_acceleration_head', 'top_m1_acceleration_body', 'top_m1_acceleration_centroid', 'top_m1_nose_x', 'top_m1_nose_y', 'top_m1_right_ear_x', 'top_m1_right_ear_y', 'top_m1_left_ear_x', 'top_m1_left_ear_y', 'top_m1_neck_x', 'top_m1_neck_y', 'top_m1_right_side_x', 'top_m1_right_side_y', 'top_m1_left_side_x', 'top_m1_left_side_y', 'top_m1_tail_base_x', 'top_m1_tail_base_y', 'top_m1_centroid_x', 'top_m1_centroid_y', 'top_m1_centroid_head_x', 'top_m1_centroid_head_y', 'top_m1_centroid_body_x', 'top_m1_centroid_body_y', 'top_m1_dist_nose_right_ear', 'top_m1_dist_nose_left_ear', 'top_m1_dist_nose_neck', 'top_m1_dist_nose_right_side', 'top_m1_dist_nose_left_side', 'top_m1_dist_nose_tail_base', 'top_m1_dist_right_ear_nose', 'top_m1_dist_right_ear_left_ear', 'top_m1_dist_right_ear_neck', 'top_m1_dist_right_ear_right_side', 'top_m1_dist_right_ear_left_side', 'top_m1_dist_right_ear_tail_base', 'top_m1_dist_left_ear_nose', 'top_m1_dist_left_ear_right_ear', 'top_m1_dist_left_ear_neck', 'top_m1_dist_left_ear_right_side', 'top_m1_dist_left_ear_left_side', 'top_m1_dist_left_ear_tail_base', 'top_m1_dist_neck_nose', 'top_m1_dist_neck_right_ear', 'top_m1_dist_neck_left_ear', 'top_m1_dist_neck_right_side', 'top_m1_dist_neck_left_side', 'top_m1_dist_neck_tail_base', 'top_m1_dist_right_side_nose', 'top_m1_dist_right_side_right_ear', 'top_m1_dist_right_side_left_ear', 'top_m1_dist_right_side_neck', 'top_m1_dist_right_side_left_side', 'top_m1_dist_right_side_tail_base', 'top_m1_dist_left_side_nose', 'top_m1_dist_left_side_right_ear', 'top_m1_dist_left_side_left_ear', 'top_m1_dist_left_side_neck', 'top_m1_dist_left_side_right_side', 'top_m1_dist_left_side_tail_base', 'top_m1_dist_tail_base_nose', 'top_m1_dist_tail_base_right_ear', 'top_m1_dist_tail_base_left_ear', 'top_m1_dist_tail_base_neck', 'top_m1_dist_tail_base_right_side', 'top_m1_dist_tail_base_left_side', 'top_m0m1_angle_between', 'top_m0m1_facing_angle', 'top_m0m1_angle_social', 'top_m0m1_radial_vel', 'top_m0m1_tangential_vel', 'top_m0m1_area_ellipse_ratio', 'top_m0m1_dist_centroid', 'top_m0m1_dist_nose', 'top_m0m1_dist_head', 'top_m0m1_dist_body', 'top_m0m1_dist_head_body', 'top_m0m1_dist_gap', 'top_m0m1_overlap_bboxes', 'top_m0m1_dist_m1nose_m2nose', 'top_m0m1_dist_m1nose_m2right_ear', 'top_m0m1_dist_m1nose_m2left_ear', 'top_m0m1_dist_m1nose_m2neck', 'top_m0m1_dist_m1nose_m2right_side', 'top_m0m1_dist_m1nose_m2left_side', 'top_m0m1_dist_m1nose_m2tail_base', 'top_m0m1_dist_m1right_ear_m2nose', 'top_m0m1_dist_m1right_ear_m2right_ear', 'top_m0m1_dist_m1right_ear_m2left_ear', 'top_m0m1_dist_m1right_ear_m2neck', 'top_m0m1_dist_m1right_ear_m2right_side', 'top_m0m1_dist_m1right_ear_m2left_side', 'top_m0m1_dist_m1right_ear_m2tail_base', 'top_m0m1_dist_m1left_ear_m2nose', 'top_m0m1_dist_m1left_ear_m2right_ear', 'top_m0m1_dist_m1left_ear_m2left_ear', 'top_m0m1_dist_m1left_ear_m2neck', 'top_m0m1_dist_m1left_ear_m2right_side', 'top_m0m1_dist_m1left_ear_m2left_side', 'top_m0m1_dist_m1left_ear_m2tail_base', 'top_m0m1_dist_m1neck_m2nose', 'top_m0m1_dist_m1neck_m2right_ear', 'top_m0m1_dist_m1neck_m2left_ear', 'top_m0m1_dist_m1neck_m2neck', 'top_m0m1_dist_m1neck_m2right_side', 'top_m0m1_dist_m1neck_m2left_side', 'top_m0m1_dist_m1neck_m2tail_base', 'top_m0m1_dist_m1right_side_m2nose', 'top_m0m1_dist_m1right_side_m2right_ear', 'top_m0m1_dist_m1right_side_m2left_ear', 'top_m0m1_dist_m1right_side_m2neck', 'top_m0m1_dist_m1right_side_m2right_side', 'top_m0m1_dist_m1right_side_m2left_side', 'top_m0m1_dist_m1right_side_m2tail_base', 'top_m0m1_dist_m1left_side_m2nose', 'top_m0m1_dist_m1left_side_m2right_ear', 'top_m0m1_dist_m1left_side_m2left_ear', 'top_m0m1_dist_m1left_side_m2neck', 'top_m0m1_dist_m1left_side_m2right_side', 'top_m0m1_dist_m1left_side_m2left_side', 'top_m0m1_dist_m1left_side_m2tail_base', 'top_m0m1_dist_m1tail_base_m2nose', 'top_m0m1_dist_m1tail_base_m2right_ear', 'top_m0m1_dist_m1tail_base_m2left_ear', 'top_m0m1_dist_m1tail_base_m2neck', 'top_m0m1_dist_m1tail_base_m2right_side', 'top_m0m1_dist_m1tail_base_m2left_side', 'top_m0m1_dist_m1tail_base_m2tail_base', 'top_m1m0_angle_between', 'top_m1m0_facing_angle', 'top_m1m0_angle_social', 'top_m1m0_radial_vel', 'top_m1m0_tangential_vel', 'top_m1m0_area_ellipse_ratio', 'top_m1m0_dist_centroid', 'top_m1m0_dist_nose', 'top_m1m0_dist_head', 'top_m1m0_dist_body', 'top_m1m0_dist_head_body', 'top_m1m0_dist_gap', 'top_m1m0_overlap_bboxes', 'top_m1m0_dist_m1nose_m2nose', 'top_m1m0_dist_m1nose_m2right_ear', 'top_m1m0_dist_m1nose_m2left_ear', 'top_m1m0_dist_m1nose_m2neck', 'top_m1m0_dist_m1nose_m2right_side', 'top_m1m0_dist_m1nose_m2left_side', 'top_m1m0_dist_m1nose_m2tail_base', 'top_m1m0_dist_m1right_ear_m2nose', 'top_m1m0_dist_m1right_ear_m2right_ear', 'top_m1m0_dist_m1right_ear_m2left_ear', 'top_m1m0_dist_m1right_ear_m2neck', 'top_m1m0_dist_m1right_ear_m2right_side', 'top_m1m0_dist_m1right_ear_m2left_side', 'top_m1m0_dist_m1right_ear_m2tail_base', 'top_m1m0_dist_m1left_ear_m2nose', 'top_m1m0_dist_m1left_ear_m2right_ear', 'top_m1m0_dist_m1left_ear_m2left_ear', 'top_m1m0_dist_m1left_ear_m2neck', 'top_m1m0_dist_m1left_ear_m2right_side', 'top_m1m0_dist_m1left_ear_m2left_side', 'top_m1m0_dist_m1left_ear_m2tail_base', 'top_m1m0_dist_m1neck_m2nose', 'top_m1m0_dist_m1neck_m2right_ear', 'top_m1m0_dist_m1neck_m2left_ear', 'top_m1m0_dist_m1neck_m2neck', 'top_m1m0_dist_m1neck_m2right_side', 'top_m1m0_dist_m1neck_m2left_side', 'top_m1m0_dist_m1neck_m2tail_base', 'top_m1m0_dist_m1right_side_m2nose', 'top_m1m0_dist_m1right_side_m2right_ear', 'top_m1m0_dist_m1right_side_m2left_ear', 'top_m1m0_dist_m1right_side_m2neck', 'top_m1m0_dist_m1right_side_m2right_side', 'top_m1m0_dist_m1right_side_m2left_side', 'top_m1m0_dist_m1right_side_m2tail_base', 'top_m1m0_dist_m1left_side_m2nose', 'top_m1m0_dist_m1left_side_m2right_ear', 'top_m1m0_dist_m1left_side_m2left_ear', 'top_m1m0_dist_m1left_side_m2neck', 'top_m1m0_dist_m1left_side_m2right_side', 'top_m1m0_dist_m1left_side_m2left_side', 'top_m1m0_dist_m1left_side_m2tail_base', 'top_m1m0_dist_m1tail_base_m2nose', 'top_m1m0_dist_m1tail_base_m2right_ear', 'top_m1m0_dist_m1tail_base_m2left_ear', 'top_m1m0_dist_m1tail_base_m2neck', 'top_m1m0_dist_m1tail_base_m2right_side', 'top_m1m0_dist_m1tail_base_m2left_side', 'top_m1m0_dist_m1tail_base_m2tail_base']


In [5]:
t.keys()

dict_keys(['feature_names', 'sequences', 'vocabulary'])

In [10]:
t["feature_names"]

['top_m0_centroid_x',
 'top_m0_centroid_y',
 'top_m0_centroid_head_x',
 'top_m0_centroid_head_y',
 'top_m0_centroid_body_x',
 'top_m0_centroid_body_y',
 'top_m0_dist_nose_right_ear',
 'top_m0_dist_nose_left_ear',
 'top_m0_dist_nose_neck',
 'top_m0_dist_nose_right_side',
 'top_m0_dist_nose_left_side',
 'top_m0_dist_nose_tail_base',
 'top_m0_dist_right_ear_nose',
 'top_m0_dist_right_ear_left_ear',
 'top_m0_dist_right_ear_neck',
 'top_m0_dist_right_ear_right_side',
 'top_m0_dist_right_ear_left_side',
 'top_m0_dist_right_ear_tail_base',
 'top_m0_dist_left_ear_nose',
 'top_m0_dist_left_ear_right_ear',
 'top_m0_dist_left_ear_neck',
 'top_m0_dist_left_ear_right_side',
 'top_m0_dist_left_ear_left_side',
 'top_m0_dist_left_ear_tail_base',
 'top_m0_dist_neck_nose',
 'top_m0_dist_neck_right_ear',
 'top_m0_dist_neck_left_ear',
 'top_m0_dist_neck_right_side',
 'top_m0_dist_neck_left_side',
 'top_m0_dist_neck_tail_base',
 'top_m0_dist_right_side_nose',
 'top_m0_dist_right_side_right_ear',
 'top_m0_d

In [7]:
np.asarray(t["sequences"]["mabe_challenge"]['1cac195d39'][0]["features"])[0, 0, :]

array([ 0.45253696,  0.48522699,  0.54786353, -1.71051018,  1.46158758,
        6.58019989,  2.68156417,  2.45386627, 55.43411956,  7.21932362,
        4.07895521,  4.07895521,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        , 16.31171185,  7.67872021,
       14.5659399 , 10.62471039, 13.58394317,  7.56960946, 13.3111663 ,
        9.26082605,  9.92873313, 13.13425758,  7.9101843 ,  9.1517153 ,
        6.8736322 , 13.18881296, 11.78361584, 10.08695028, 14.82053164,
        8.62434669,  8.23751655, 11.82492861,  3.42440913,  2.72995003,
        3.39209854,  8.39674374,  8.52967641, 10.92879085,  3.42440913,
        3.20904335,  1.85327744,  5.27271407,  6.81680255,  8.10840426,
        2.72995003,  3.20904335,  1.71307348,  6.65776758,  5.8902121 ,
        8.75235518,  3.39209854,  1.85327744,  1.71307348,  5.14240469,
        5.40208401,  7.54128148,  8.39674374,  5.27271407,  6.65776758,
        5.14240469,  4.46488324,  3.05558799,  8.52967641,  6.81

In [8]:
np.asarray(features)[np.asarray(t["sequences"]["mabe_challenge"]['1cac195d39'][0]["features"])[0, 2000, :] == 0]

array(['top_m0m1_dist_scaled', 'top_m0m1_overlap_bboxes',
       'top_m1m0_dist_scaled', 'top_m1m0_overlap_bboxes'], dtype='<U39')

In [20]:
np.sum(np.asarray(t["sequences"]["mabe_challenge"]['1cac195d39'][0]["features"])[0, :, np.asarray(features) == 'top_m0_dist_edge_x'])

0.0

In [23]:
features

['top_m0_phi',
 'top_m0_ori_head',
 'top_m0_ori_body',
 'top_m0_angle_head_body_l',
 'top_m0_angle_head_body_r',
 'top_m0_major_axis_len',
 'top_m0_minor_axis_len',
 'top_m0_axis_ratio',
 'top_m0_area_ellipse',
 'top_m0_dist_edge_x',
 'top_m0_dist_edge_y',
 'top_m0_dist_edge',
 'top_m0_speed',
 'top_m0_speed_centroid',
 'top_m0_speed_fwd',
 'top_m0_acceleration_head',
 'top_m0_acceleration_body',
 'top_m0_acceleration_centroid',
 'top_m0_nose_x',
 'top_m0_nose_y',
 'top_m0_right_ear_x',
 'top_m0_right_ear_y',
 'top_m0_left_ear_x',
 'top_m0_left_ear_y',
 'top_m0_neck_x',
 'top_m0_neck_y',
 'top_m0_right_side_x',
 'top_m0_right_side_y',
 'top_m0_left_side_x',
 'top_m0_left_side_y',
 'top_m0_tail_base_x',
 'top_m0_tail_base_y',
 'top_m0_centroid_x',
 'top_m0_centroid_y',
 'top_m0_centroid_head_x',
 'top_m0_centroid_head_y',
 'top_m0_centroid_body_x',
 'top_m0_centroid_body_y',
 'top_m0_dist_nose_right_ear',
 'top_m0_dist_nose_left_ear',
 'top_m0_dist_nose_neck',
 'top_m0_dist_nose_right_s

array(['top_m0_phi', 'top_m0_ori_head', 'top_m0_ori_body',
       'top_m0_angle_head_body_l', 'top_m0_angle_head_body_r',
       'top_m0_major_axis_len', 'top_m0_minor_axis_len',
       'top_m0_axis_ratio', 'top_m0_area_ellipse', 'top_m0_dist_edge_x',
       'top_m0_dist_edge_y', 'top_m0_dist_edge', 'top_m0_speed',
       'top_m0_speed_centroid', 'top_m0_speed_fwd',
       'top_m0_acceleration_head', 'top_m0_acceleration_body',
       'top_m0_acceleration_centroid', 'top_m0_nose_x', 'top_m0_nose_y',
       'top_m0_right_ear_x', 'top_m0_right_ear_y', 'top_m0_left_ear_x',
       'top_m0_left_ear_y', 'top_m0_neck_x', 'top_m0_neck_y',
       'top_m0_right_side_x', 'top_m0_right_side_y', 'top_m0_left_side_x',
       'top_m0_left_side_y', 'top_m0_tail_base_x', 'top_m0_tail_base_y',
       'top_m0_centroid_x', 'top_m0_centroid_y', 'top_m0_centroid_head_x',
       'top_m0_centroid_head_y', 'top_m0_centroid_body_x',
       'top_m0_centroid_body_y', 'top_m0_dist_nose_right_ear',
       'top_m0_d